# Training BIANCA Automatically

The purpose of this notebook is to document the process to train BIANCA and to have a script that allows re-doing the process automatically without having to manually sort, rename or adjust any files.

Before proceeding to the code part, it is essential to understand the steps that will be required for training BIANCA.

Following the training practical [https://open.win.ox.ac.uk/pages/fslcourse/practicals/seg_struc/index.html#bianca], we will be needing the following files for each subject, regardless of how many subjects there are:
- Flair Brain Image
- Flair Brian Lesion Mask 
- T1 Brain Image (Optional)
- Flair Brain to MNI for .mat (Optional)
- T1 Brain to Flair Brain (Optional)

The terms 'Optional' are used so that, in the case of good quality Brain Images, we can use the additional files as paramters to BIANCA to support the Model in its output.

## Pre-requisities:

To start off, we will be needing the **Flair Brain Image**, **Flair Brain Lesion Mask**, and the **T1 Brain Image** (Optional) for each subject. 
Once we have those, we can proceed and extract the remaining by registration and resampling. 

Another condition to ensure the proper functioning of the code blocks in this Jupyter Notebook is to have a folder that contains the names of the **Flair Brain Image**, **Flair Brain Lesion Mask**, and the **T1 Brain Image** with the following convention and structure:

**ms_0xx_tpy_z.nii.gz**
(x = subject number, y = timepoint, z = file name)
- As an example:
    - ms_002_tp1_flair_brain.nii.gz 
        - (having 'flair_brain' in the name is necessary)
    - ms_002_tp1_lesion_mask.nii.gz 
        - (having 'mask' in the name is necessary)
    - ms_002_tp1_T1_brain.nii.gz    
        - (having 'T1' in the name is necessary)

### An example of a folder I created for the test run is as follows:


<img src="notebook_images/file_structure.png" alt="Drawing" style="width: 800px;"/>


## First Step: Resampling all Images

One thing to note is that BIANCA requires all the **Flair Brain Image(s)** to be in the same dimensions, therefore we will figure out the most common/ occuring dimensions and will then resample all other images to be of that very dimension.

### Getting the dimensions for all the flair brain Images

#### First, we get the folder path where all the images are stored

In [1]:
from tkinter import Tk, filedialog

root = Tk() # pointing root to Tk() to use it as Tk() in program.
root.withdraw() # Hides small tkinter window.
''
root.attributes('-topmost', True) # Opened windows will be active. above all windows despite of selection.
''
open_file = filedialog.askdirectory() # Returns opened path as str
print(open_file) 

/Volumes/MS_training/Automated BIANCA Training/All_Files


#### Ensuring the correct folder is selected by printing out the names of all the files within the folder

In [2]:
import os
files = os.listdir(open_file) # returns list
print(files)

['.DS_Store', 'ms_002_tp1_flair_brain.nii.gz', 'ms_002_tp1_flair_lesion_mask.nii.gz', 'ms_002_tp1_T1_brain.nii.gz', 'ms_003_tp1_flair_brain.nii.gz', 'ms_003_tp1_flair_lesion_mask.nii.gz', 'ms_003_tp1_T1_brain.nii.gz', 'ms_003_tp6_flair_brain.nii.gz', 'ms_003_tp6_flair_lesion_mask.nii.gz', 'ms_003_tp6_T1_brain.nii.gz', 'ms_004_tp1_flair_brain.nii.gz', 'ms_004_tp1_flair_lesion_mask.nii.gz', 'ms_004_tp1_T1_brain.nii.gz', 'ms_004_tp7_flair_brain.nii.gz', 'ms_004_tp7_flair_lesion_mask.nii.gz', 'ms_004_tp7_T1_brain.nii.gz', 'ms_006_tp1_flair_brain.nii.gz', 'ms_006_tp1_flair_lesion_mask.nii.gz', 'ms_006_tp1_T1_brain.nii.gz', 'ms_006_tp3_flair_brain.nii.gz', 'ms_006_tp3_flair_lesion_mask.nii.gz', 'ms_006_tp3_T1_brain.nii.gz', 'ms_007_tp1_flair_brain.nii.gz', 'ms_007_tp1_flair_lesion_mask.nii.gz', 'ms_007_tp1_T1_brain.nii.gz', 'ms_007_tp2_flair_brain.nii.gz', 'ms_007_tp2_flair_lesion_mask.nii.gz', 'ms_007_tp2_T1_brain.nii.gz', 'ms_008_tp1_flair_brain.nii.gz', 'ms_008_tp1_flair_lesion_mask.nii.g

#### Second, we create 3 different lists, and then store the files for the Flair Brain, Lesion Mask and T1 Brain images respectively

In [3]:
flairs = []
masks = []
t1s = []

for file in files:
#     print(file)
    if 'flair_brain' in file:
        flairs.append(file)
    elif 'mask' in file:
        masks.append(file)
    elif 'T1' in file:
        t1s.append(file)

In [4]:
print("All the Flair Brain Images are:")
print(flairs)
print()
print("All the Lesion Mask Images are:")
print(masks)
print()
print("All the T1 Brain Images are:")
print(t1s)

All the Flair Brain Images are:
['ms_002_tp1_flair_brain.nii.gz', 'ms_003_tp1_flair_brain.nii.gz', 'ms_003_tp6_flair_brain.nii.gz', 'ms_004_tp1_flair_brain.nii.gz', 'ms_004_tp7_flair_brain.nii.gz', 'ms_006_tp1_flair_brain.nii.gz', 'ms_006_tp3_flair_brain.nii.gz', 'ms_007_tp1_flair_brain.nii.gz', 'ms_007_tp2_flair_brain.nii.gz', 'ms_008_tp1_flair_brain.nii.gz', 'ms_008_tp6_flair_brain.nii.gz', 'ms_009_tp1_flair_brain.nii.gz', 'ms_009_tp4_flair_brain.nii.gz', 'ms_010_tp1_flair_brain.nii.gz', 'ms_011_tp1_flair_brain.nii.gz', 'ms_012_tp1_flair_brain.nii.gz', 'ms_013_tp1_flair_brain.nii.gz']

All the Lesion Mask Images are:
['ms_002_tp1_flair_lesion_mask.nii.gz', 'ms_003_tp1_flair_lesion_mask.nii.gz', 'ms_003_tp6_flair_lesion_mask.nii.gz', 'ms_004_tp1_flair_lesion_mask.nii.gz', 'ms_004_tp7_flair_lesion_mask.nii.gz', 'ms_006_tp1_flair_lesion_mask.nii.gz', 'ms_006_tp3_flair_lesion_mask.nii.gz', 'ms_007_tp1_flair_lesion_mask.nii.gz', 'ms_007_tp2_flair_lesion_mask.nii.gz', 'ms_008_tp1_flair_les

### Checking the most frequently occuring dimension

In [5]:
dimensions = []
directory_1 = open_file.replace(' ', '\ ')

def check_dimensions(file_name, printvals):
    p = os.popen(
        f"cd {directory_1} && fslinfo {file_name}")
    if p:
        output = p.read()
        #print(output)
        output = output.replace("\t", "\n")
        file_info = output.split("\n")
        while '' in file_info:
            file_info.remove('')
        #print(file_info)
        dim1 = file_info[3]
        dim2 = file_info[5]
        dim3 = file_info[7]
        if printvals== True:
            print("Dimensions of: ", file_name)
            print(dim1,dim2,dim3)
        return(dim1,dim2,dim3)

for i in range(len(flairs)):
    dimensions.append(check_dimensions(flairs[i], True))

Dimensions of:  ms_002_tp1_flair_brain.nii.gz
256 256 40
Dimensions of:  ms_003_tp1_flair_brain.nii.gz
320 320 40
Dimensions of:  ms_003_tp6_flair_brain.nii.gz
256 256 49
Dimensions of:  ms_004_tp1_flair_brain.nii.gz
256 256 25
Dimensions of:  ms_004_tp7_flair_brain.nii.gz
320 320 40
Dimensions of:  ms_006_tp1_flair_brain.nii.gz
320 320 44
Dimensions of:  ms_006_tp3_flair_brain.nii.gz
256 256 44
Dimensions of:  ms_007_tp1_flair_brain.nii.gz
256 256 43
Dimensions of:  ms_007_tp2_flair_brain.nii.gz
256 256 40
Dimensions of:  ms_008_tp1_flair_brain.nii.gz
256 256 40
Dimensions of:  ms_008_tp6_flair_brain.nii.gz
256 256 40
Dimensions of:  ms_009_tp1_flair_brain.nii.gz
256 256 40
Dimensions of:  ms_009_tp4_flair_brain.nii.gz
256 256 44
Dimensions of:  ms_010_tp1_flair_brain.nii.gz
256 256 44
Dimensions of:  ms_011_tp1_flair_brain.nii.gz
256 256 40
Dimensions of:  ms_012_tp1_flair_brain.nii.gz
256 256 40
Dimensions of:  ms_013_tp1_flair_brain.nii.gz
256 256 46


In [6]:
def most_frequent(List):
    return max(set(List), key = List.count)
 
most_freq = most_frequent(dimensions)
print("The Most Frequently Occuring Dimension is: ")
print(most_freq[0],"x", most_freq[1],"x", most_freq[2])

The Most Frequently Occuring Dimension is: 
256 x 256 x 40


### Picking an Image with the most frequently occuring dimension for reference purposes

#### Index for the most frequently occuring dimension

In [7]:
ind = dimensions.index(most_freq)

#### Getting the corresponding Value from the *flairs* list for the corresponding image

In [8]:
ref_img = flairs[ind]
print(ref_img)

ms_002_tp1_flair_brain.nii.gz


### Create a subdirectory 'Training-{DATE}' to store all the resampled files, which shall be used for training purposes

In [9]:
from datetime import date
today = date.today()
directory_name = 'Training-' + str(today)
training_directory_name = directory_name

In [10]:
path = os.path.join(open_file, directory_name)
try:
    os.mkdir(path)
except OSError as error:
    print(error)

### Within the subdirectory, we create multiple subdirectories for each subject

In [11]:
folder_names = []

for file in flairs:
    print("_".join(file.split('_')[0:3]))
    folder_name = "_".join(file.split('_')[0:3])
    folder_names.append(folder_name)
    
    path = os.path.join(open_file + '/' + directory_name, folder_name)
    try:
        os.mkdir(path)
    except OSError as error:
        print(error)

ms_002_tp1
ms_003_tp1
ms_003_tp6
ms_004_tp1
ms_004_tp7
ms_006_tp1
ms_006_tp3
ms_007_tp1
ms_007_tp2
ms_008_tp1
ms_008_tp6
ms_009_tp1
ms_009_tp4
ms_010_tp1
ms_011_tp1
ms_012_tp1
ms_013_tp1


### Now, we should essentially have a directory structure like the following:

<img src="notebook_images/new_folders.png" alt="Drawing" style="width: 700px;"/>


### Using the reference Image, we iterate through all the flair brain images with different dimensions and resample them to be in the same dimensions, and then store them into their respective subdirectory

#### We do the same for the corresponding Lesion Masks to those Flair Brain Images since they should be resampled into the same dimensions as their corresponding flairs

#### However, out of all the subjects, one of the Subjects will be solely for testing purposes and will not have an existing flair lesion mask. Therefore, since it can be any, we will ask the user for the subject number and timepoint for the test subject

In [12]:
training_folder = input("Please enter the Test Subject as ms_0xx_tpx (example: ms_003_tp1) ")

Please enter the Test Subject as ms_0xx_tpx (example: ms_003_tp1) ms_013_tp1


In [13]:
training_subject_bianca = training_folder
print("The Test Subject is: ", training_subject_bianca)

The Test Subject is:  ms_013_tp1


In [14]:
for file in flairs:
    
    folder_name = "_".join(file.split('_')[0:3])
    
    # For all the images that have a dimension other than the most occuring dimension
    if check_dimensions(file, False) != check_dimensions(ref_img, False):
        print("RESAMPLING....")
        print("FILE TO BE RESAMPLED: ", file)
        print()
                
        # Resampling the Flair Brain Image
        p = os.popen(
            f"cd {directory_1} && flirt -in {file} -ref {ref_img} -out {directory_name}/{folder_name}/{folder_name}_flair_brain.nii.gz -omat {directory_name}/{folder_name}/resampling.mat -interp nearestneighbour -dof 6")
        if p:
            output = p.read()
            print(output)
        
        if(folder_name != training_folder):
            
            # FIND THE LESION MASK FOR THE SAME FILE
            for mask in masks:
                if (folder_name in mask):
                    print("Mask to be Resampled: ",mask)
                    mask_file = mask
                    break


            # Resampling the Flair Lesion Mask
            p = os.popen(
                f"cd {directory_1} && flirt -in {mask_file} -ref {ref_img} -out {directory_name}/{folder_name}/{folder_name}_flair_lesion_mask.nii.gz -applyxfm -init {directory_name}/{folder_name}/resampling.mat -interp nearestneighbour -dof 6")
            if p:
                output = p.read()
                print(output)

        # Deleting the Resampling.mat file since it is of no use to us        
        p = os.popen(
            f"cd {directory_1} && rm {directory_name}/{folder_name}/resampling.mat")
        if p:
            output = p.read()
            print(output)
    
    # In the case that the file does not need to be resampled, we simply just copy/paste it into the specific training directory     
    else:    
        print("COPYING...")
        print("Copying the file:", file)
        # Copying the Flair Brain Images        
        p = os.popen(
            f"cd {directory_1} && cp {file} {directory_name}/{folder_name}/{folder_name}_flair_brain.nii.gz")
        if p:
            output = p.read()
            print(output)  
        
        if(folder_name != training_folder):
            
            # FIND THE LESION MASK FOR THE SAME FILE
            for mask in masks:
                if (folder_name in mask):
                    print("Mask to be Copied: ",mask)
                    mask_file = mask
                    break

            print("Copying the file:", mask_file)
            # Copying the Flair Lesion Mask        
            p = os.popen(
                f"cd {directory_1} && cp {mask_file} {directory_name}/{folder_name}/{folder_name}_flair_lesion_mask.nii.gz")
            if p:
                output = p.read()
                print(output)  
            
            
    

COPYING...
Copying the file: ms_002_tp1_flair_brain.nii.gz

Mask to be Copied:  ms_002_tp1_flair_lesion_mask.nii.gz
Copying the file: ms_002_tp1_flair_lesion_mask.nii.gz

RESAMPLING....
FILE TO BE RESAMPLED:  ms_003_tp1_flair_brain.nii.gz


Mask to be Resampled:  ms_003_tp1_flair_lesion_mask.nii.gz


RESAMPLING....
FILE TO BE RESAMPLED:  ms_003_tp6_flair_brain.nii.gz


Mask to be Resampled:  ms_003_tp6_flair_lesion_mask.nii.gz


RESAMPLING....
FILE TO BE RESAMPLED:  ms_004_tp1_flair_brain.nii.gz


Mask to be Resampled:  ms_004_tp1_flair_lesion_mask.nii.gz


RESAMPLING....
FILE TO BE RESAMPLED:  ms_004_tp7_flair_brain.nii.gz


Mask to be Resampled:  ms_004_tp7_flair_lesion_mask.nii.gz


RESAMPLING....
FILE TO BE RESAMPLED:  ms_006_tp1_flair_brain.nii.gz


Mask to be Resampled:  ms_006_tp1_flair_lesion_mask.nii.gz


RESAMPLING....
FILE TO BE RESAMPLED:  ms_006_tp3_flair_brain.nii.gz


Mask to be Resampled:  ms_006_tp3_flair_lesion_mask.nii.gz


RESAMPLING....
FILE TO BE RESAMPLED:  ms_00

## Second Step: Flair Brain to MNI for .mat (Optional)

(This step is an Optional step, and the output is only used when the Flair Brain Images are of very good quality. While we will proceed with the step, we may not be using its results in the training and testing of BIANCA).

Now that we have all the images over the same dimensions, we can move forward with the step of extracting the .mat
file needed to train BIANCA by registering the flair brain image over the sample MNI Image.

#### As a pre-requisite to this step, we need the MNI Space reference file in order to register the resampled flair brains to MNI Space and retrieve the .mat file

<img src="notebook_images/MNI_file.png" alt="Drawing" style="width: 500px;"/>


### Since the location of this file is not fixed, we can go ahead and select the file path for the MNI Space file by running the code block below:

In [17]:
from tkinter import Tk, filedialog

root = Tk() # pointing root to Tk() to use it as Tk() in program.
root.withdraw() # Hides small tkinter window.
''
root.attributes('-topmost', True) # Opened windows will be active. above all windows despite of selection.
''
mni_file = filedialog.askopenfilename() # Returns opened path as str

mni_directory = mni_file.replace(' ', '\ ')
print(mni_directory) 


/Users/ms/Documents/GitHub/MS-Longitudinal-Analysis/Fall\ 2022/BIANCA_Training_Notebook/MNI152_2mm_brain.nii.gz


### Then, we actually move forward and apply the registration command to transform the Flair Brain Image to the MNI Space and extract the .mat file

In [18]:
for file in flairs:
    
    folder_name = "_".join(file.split('_')[0:3])
    
    # Registering the FLAIR Brain Image to MNI Space
    print(f"File being Registered is:  {folder_name}_flair_brain.nii.gz")
    p = os.popen(
        f"cd {directory_1}/{directory_name} && flirt -in {folder_name}/{folder_name}_flair_brain.nii.gz -ref {mni_directory} -out {folder_name}/{folder_name}_flair_brain_to_MNI.nii.gz -omat {folder_name}/{folder_name}_flair_brain_to_MNI.mat -bins 256 -cost normcorr -searchrx -180 180 -searchry -180 180 -searchrz -180 180 -dof 7 -interp nearestneighbour")
    if p:
        output = p.read()
        print(output)

File being Registered is:  ms_002_tp1_flair_brain.nii.gz

File being Registered is:  ms_003_tp1_flair_brain.nii.gz

File being Registered is:  ms_003_tp6_flair_brain.nii.gz

File being Registered is:  ms_004_tp1_flair_brain.nii.gz

File being Registered is:  ms_004_tp7_flair_brain.nii.gz

File being Registered is:  ms_006_tp1_flair_brain.nii.gz

File being Registered is:  ms_006_tp3_flair_brain.nii.gz

File being Registered is:  ms_007_tp1_flair_brain.nii.gz

File being Registered is:  ms_007_tp2_flair_brain.nii.gz

File being Registered is:  ms_008_tp1_flair_brain.nii.gz

File being Registered is:  ms_008_tp6_flair_brain.nii.gz

File being Registered is:  ms_009_tp1_flair_brain.nii.gz

File being Registered is:  ms_009_tp4_flair_brain.nii.gz

File being Registered is:  ms_010_tp1_flair_brain.nii.gz

File being Registered is:  ms_011_tp1_flair_brain.nii.gz

File being Registered is:  ms_012_tp1_flair_brain.nii.gz

File being Registered is:  ms_013_tp1_flair_brain.nii.gz



## Third Step: T1 Brain to Flair Brain (Optional)

(As with the Regsitration to MNI Space, this is again an Optional Step and the output is only considered for good quality images)

Following a similar approach to the Second Step, this time we will use the T1 Brain Image and then register it onto the Flair Brain images across all the subjects. These T1 Images (stored in the master directory, will actually be registered to the resampled version of the Flair Brain Images (stored in their respective directories).

As a demonstration, the grey highlighted file ***'ms_003_tp1_T1_brain.nii.gz'*** in the left most column will be registered to the blue highlighted file ***'ms_003_tp1_flair_brain.nii.gz'*** in the right most column.

<img src="notebook_images/t1_flair.png" alt="Drawing" style="width: 1000px;"/>


### Now, we actually move forward and apply the registration command to transform the T1 Brain Image to the Resampled Flair Brain Image and obtain the T1 Brain to Flair Brain Image

In [19]:
for file in t1s:
    
    folder_name = "_".join(file.split('_')[0:3])
    
    # Registering the T1 Image to Flair Image
    print("Registering the file: ",file)
    p = os.popen(
        f"cd {directory_1} && flirt -in {file} -ref {directory_name}/{folder_name}/{folder_name}_flair_brain.nii.gz -out {directory_name}/{folder_name}/{folder_name}_t1_to_flair.nii.gz -omat {directory_name}/{folder_name}/{folder_name}_t1_to_flair.mat -bins 256 -cost normcorr -searchrx -180 180 -searchry -180 180 -searchrz -180 180 -dof 7 -interp nearestneighbour")
    if p:
        output = p.read()
        print(output)

Registering the file:  ms_002_tp1_T1_brain.nii.gz

Registering the file:  ms_003_tp1_T1_brain.nii.gz

Registering the file:  ms_003_tp6_T1_brain.nii.gz

Registering the file:  ms_004_tp1_T1_brain.nii.gz

Registering the file:  ms_004_tp7_T1_brain.nii.gz

Registering the file:  ms_006_tp1_T1_brain.nii.gz

Registering the file:  ms_006_tp3_T1_brain.nii.gz

Registering the file:  ms_007_tp1_T1_brain.nii.gz

Registering the file:  ms_007_tp2_T1_brain.nii.gz

Registering the file:  ms_008_tp1_T1_brain.nii.gz

Registering the file:  ms_008_tp6_T1_brain.nii.gz

Registering the file:  ms_009_tp1_T1_brain.nii.gz

Registering the file:  ms_009_tp4_T1_brain.nii.gz

Registering the file:  ms_010_tp1_T1_brain.nii.gz

Registering the file:  ms_011_tp1_T1_brain.nii.gz

Registering the file:  ms_012_tp1_T1_brain.nii.gz

Registering the file:  ms_013_tp1_T1_brain.nii.gz



## So far, we should have the following files and a similar file structure:

<img src="notebook_images/all_file_structure.png" alt="Drawing" style="width: 800px;"/>

## Fourth Step: Generating the Masterfile

Once we have all the files, we will be creating a masterfile.txt which includes all the four files that were highlighted on the previous screenshot, or only the Flair Brain & Lesion Mask, whichever is suitable for us.

For the purposes of our use, we will go ahead with the Flair Brain & Lesion masks only, as follows:

In [15]:
f = open(f"{open_file}/{directory_name}/masterfile.txt", "w")

for file in flairs:
    
    folder_name = "_".join(file.split('_')[0:3])
    
    if(folder_name != training_folder):

        print(f"{folder_name}/{folder_name}_flair_brain.nii.gz {folder_name}/{folder_name}_flair_lesion_mask.nii.gz")

        f.write(f"{folder_name}/{folder_name}_flair_brain.nii.gz {folder_name}/{folder_name}_flair_lesion_mask.nii.gz \n")

# Writing the test subject at the very end to correspond with the Training Command (in the upcoming code blocks)
print(f"{training_folder}/{training_folder}_flair_brain.nii.gz {training_folder}/{training_folder}_flair_lesion_mask.nii.gz")

f.write(f"{training_folder}/{training_folder}_flair_brain.nii.gz {training_folder}/{training_folder}_flair_lesion_mask.nii.gz \n")

        
f.close()

ms_002_tp1/ms_002_tp1_flair_brain.nii.gz ms_002_tp1/ms_002_tp1_flair_lesion_mask.nii.gz
ms_003_tp1/ms_003_tp1_flair_brain.nii.gz ms_003_tp1/ms_003_tp1_flair_lesion_mask.nii.gz
ms_003_tp6/ms_003_tp6_flair_brain.nii.gz ms_003_tp6/ms_003_tp6_flair_lesion_mask.nii.gz
ms_004_tp1/ms_004_tp1_flair_brain.nii.gz ms_004_tp1/ms_004_tp1_flair_lesion_mask.nii.gz
ms_004_tp7/ms_004_tp7_flair_brain.nii.gz ms_004_tp7/ms_004_tp7_flair_lesion_mask.nii.gz
ms_006_tp1/ms_006_tp1_flair_brain.nii.gz ms_006_tp1/ms_006_tp1_flair_lesion_mask.nii.gz
ms_006_tp3/ms_006_tp3_flair_brain.nii.gz ms_006_tp3/ms_006_tp3_flair_lesion_mask.nii.gz
ms_007_tp1/ms_007_tp1_flair_brain.nii.gz ms_007_tp1/ms_007_tp1_flair_lesion_mask.nii.gz
ms_007_tp2/ms_007_tp2_flair_brain.nii.gz ms_007_tp2/ms_007_tp2_flair_lesion_mask.nii.gz
ms_008_tp1/ms_008_tp1_flair_brain.nii.gz ms_008_tp1/ms_008_tp1_flair_lesion_mask.nii.gz
ms_008_tp6/ms_008_tp6_flair_brain.nii.gz ms_008_tp6/ms_008_tp6_flair_lesion_mask.nii.gz
ms_009_tp1/ms_009_tp1_flair_brai

However, the code block below could have been used for making use of the other two files, particularly the MNI Space Registration and the T1 to Flair Registration. 

In [33]:
# f = open(f"{open_file}/{directory_name}/masterfile.txt", "w")

# for file in flairs:
    
#     folder_name = "_".join(file.split('_')[0:3])
    
#     if(folder_name != training_folder):

#         print(f"{folder_name}/{folder_name}_flair_brain.nii.gz {folder_name}/{folder_name}_t1_to_flair.nii.gz {folder_name}/{folder_name}_flair_brain_to_MNI.mat {folder_name}/{folder_name}_flair_lesion_mask.nii.gz")

#         f.write(f"{folder_name}/{folder_name}_flair_brain.nii.gz {folder_name}/{folder_name}_t1_to_flair.nii.gz {folder_name}/{folder_name}_flair_brain_to_MNI.mat {folder_name}/{folder_name}_flair_lesion_mask.nii.gz \n")

# # Writing the test subject at the very end to correspond with the Training Command (in the upcoming code blocks)
# print(f"{training_folder}/{training_folder}_flair_brain.nii.gz {training_folder}/{training_folder}_t1_to_flair.nii.gz {training_folder}/{training_folder}_flair_brain_to_MNI.mat {training_folder}/{training_folder}_flair_lesion_mask.nii.gz")

# f.write(f"{training_folder}/{training_folder}_flair_brain.nii.gz {training_folder}/{training_folder}_t1_to_flair.nii.gz {training_folder}/{training_folder}_flair_brain_to_MNI.mat {training_folder}/{training_folder}_flair_lesion_mask.nii.gz \n")

        
# f.close()

### The file should look similar to the following:

Each subject should have their individual entry on each line

<img src="notebook_images/masterfile_img.png" alt="Drawing" style="width: 1000px;"/>

## Fifth Step: Training BIANCA

Now that we have all the data to Train BIANCA, and it is linked in the Masterfile, we will move forward and train BIANCA

In [16]:
training_nums = []
for i in range(len(flairs)):
    training_nums.append(i)

t_nums = ",".join([str(elem+1) for elem in training_nums])
print(t_nums)

1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17


In [17]:
print(
    f"cd {directory_1}/{directory_name} && \
    bianca --singlefile=masterfile.txt --trainingnums={t_nums} --labelfeaturenum=2 \
    --querysubjectnum={len(training_nums)} --brainmaskfeaturenum=1 \
    --trainingpts=2000 --nonlespts=10000 --selectpts=noborder -o {training_folder}/bianca_output \
    --saveclassifierdata=mytraining -v")

cd /Volumes/MS_training/Automated\ BIANCA\ Training/All_Files/Training-2022-12-04 &&     bianca --singlefile=masterfile.txt --trainingnums=1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17 --labelfeaturenum=2     --querysubjectnum=17 --brainmaskfeaturenum=1     --trainingpts=2000 --nonlespts=10000 --selectpts=noborder -o ms_013_tp1/bianca_output     --saveclassifierdata=mytraining -v


The code block below runs the Training Command for Bianca, and tests it on the very last image. There are numerous other query parameters that could be passed, which are detailed further here:
https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/BIANCA/Userguide#BIANCA_options

In [18]:
p = os.popen(
    f"cd {directory_1}/{directory_name} && \
    bianca --singlefile=masterfile.txt --trainingnums={t_nums} --labelfeaturenum=2 \
    --querysubjectnum={len(training_nums)} --brainmaskfeaturenum=1 \
    --trainingpts=2000 --nonlespts=10000 --selectpts=noborder -o {training_folder}/bianca_output \
    --saveclassifierdata=mytraining -v")
if p:
    output = p.read()
    print(output)

Number of modalities = 2 , number of possible training subjects = 17
Files are: label file list = , data file list = ['masterfile.txt']
Number of training points = 2000, mode = ['npoints', 'noborder']
no spatial features used
Using all available images as intensity features (no intensity features subset specified)
Loading training data
 ... reading label ms_002_tp1/ms_002_tp1_flair_lesion_mask.nii.gz
 ... reading image ms_002_tp1/ms_002_tp1_flair_brain.nii.gz
 ... reading label ms_003_tp1/ms_003_tp1_flair_lesion_mask.nii.gz
 ... reading image ms_003_tp1/ms_003_tp1_flair_brain.nii.gz
 ... reading label ms_003_tp6/ms_003_tp6_flair_lesion_mask.nii.gz
 ... reading image ms_003_tp6/ms_003_tp6_flair_brain.nii.gz
 ... reading label ms_004_tp1/ms_004_tp1_flair_lesion_mask.nii.gz
 ... reading image ms_004_tp1/ms_004_tp1_flair_brain.nii.gz
 ... reading label ms_004_tp7/ms_004_tp7_flair_lesion_mask.nii.gz
 ... reading image ms_004_tp7/ms_004_tp7_flair_brain.nii.gz
 ... reading label ms_006_tp1/ms

/Applications/fsl/bin/bianca:558: DeprecationWarning: get_header method is deprecated.
Please use the ``img.header`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  GlobalOpts.pixdims=imobj.get_header().get_zooms()
/Applications/fsl/bin/bianca:1026: DeprecationWarning: get_affine method is deprecated.
Please use the ``img.affine`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  pnii = nib.Nifti1Image(pvalim,queryimcl.get_affine(),header=queryimcl.get_header())
/Applications/fsl/bin/bianca:1026: DeprecationWarning: get_header method is deprecated.
Please use the ``img.header`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  pnii = nib.Nifti1Image(pvalim,queryimcl.get_affine(),header=queryimcl.get_header())


The code block commented below could be used for making use of all the 4 files that we generated, with the appropriate masterfile (the method of generation for which is also commented above)

In [32]:
# p = os.popen(
#     f"cd {directory_1}/{directory_name} && \
#     bianca --singlefile=masterfile.txt --trainingnums={t_nums} --labelfeaturenum=4 \
#     --querysubjectnum={len(training_nums)} --brainmaskfeaturenum=1 --featuresubset=1,2 --matfeaturenum=3 \
#     --trainingpts=2000 --nonlespts=10000 --selectpts=noborder -o {training_folder}/bianca_output \
#     --saveclassifierdata=mytraining -v")
# if p:
#     output = p.read()
#     print(output)

Number of modalities = 2 , number of possible training subjects = 17
Files are: label file list = , data file list = ['masterfile.txt']
Number of training points = 2000, mode = ['npoints', 'noborder']
no spatial features used
Using all available images as intensity features (no intensity features subset specified)
Loading training data
 ... reading label ms_002_tp1/ms_002_tp1_flair_lesion_mask.nii.gz
 ... reading image ms_002_tp1/ms_002_tp1_flair_brain.nii.gz
 ... reading label ms_003_tp1/ms_003_tp1_flair_lesion_mask.nii.gz
 ... reading image ms_003_tp1/ms_003_tp1_flair_brain.nii.gz
 ... reading label ms_003_tp6/ms_003_tp6_flair_lesion_mask.nii.gz
 ... reading image ms_003_tp6/ms_003_tp6_flair_brain.nii.gz
 ... reading label ms_004_tp1/ms_004_tp1_flair_lesion_mask.nii.gz
 ... reading image ms_004_tp1/ms_004_tp1_flair_brain.nii.gz
 ... reading label ms_004_tp7/ms_004_tp7_flair_lesion_mask.nii.gz
 ... reading image ms_004_tp7/ms_004_tp7_flair_brain.nii.gz
 ... reading label ms_006_tp1/ms

/Applications/fsl/bin/bianca:558: DeprecationWarning: get_header method is deprecated.
Please use the ``img.header`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  GlobalOpts.pixdims=imobj.get_header().get_zooms()
/Applications/fsl/bin/bianca:1026: DeprecationWarning: get_affine method is deprecated.
Please use the ``img.affine`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  pnii = nib.Nifti1Image(pvalim,queryimcl.get_affine(),header=queryimcl.get_header())
/Applications/fsl/bin/bianca:1026: DeprecationWarning: get_header method is deprecated.
Please use the ``img.header`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  pnii = nib.Nifti1Image(pvalim,queryimcl.get_affine(),header=queryimcl.get_header())


## If the above code block ran successfully, we have successfully ran and stored the model

### However, as a final step, we will move ahead and produce a file which contains some basic information, or metadata in proper terminology, from the training so that it could be used when we are testing using this model.

The information we need to store is the reference dimension and the reference image with that dimension

In [19]:
print(most_freq)

('256', '256', '40')


In [20]:
storage_file = open(f"{open_file}/{directory_name}/metadata.txt", "w")
storage_file.write(f"{most_freq[0]},{most_freq[1]},{most_freq[2]}\n")
storage_file.write(f"{ref_img}")

29

## Adaptive Thresholding

This step is needed to build on top of our BIANCA output. The purpose that it would serve is to allow for a dynamic way of applying a threshold, rather than a Binary method of doing so - which would have led to discrepancies in our results.

https://git.fmrib.ox.ac.uk/vaanathi/LOCATE-BIANCA

LOCATE is a supervised method and hence requires data for training. In the current version the user needs to provide data in a specific, standardized manner (which can typically be achieved by some moving and renaming of files)

### Step 1. Creating the Directory & Sub Directories for Training/ Testing

In [21]:
training_directory_name

'Training-2022-12-04'

In [22]:
print(open_file + "/" + training_directory_name)

/Volumes/MS_training/Automated BIANCA Training/All_Files/Training-2022-12-04


In [23]:
from datetime import date
today = date.today()
directory_name = 'LOCATE_Training-' + str(today)

# Creating the Parent Directory
path = os.path.join(open_file + "/" + training_directory_name, directory_name)
try:
    os.mkdir(path)
except OSError as error:
    print(error)
    
# Creating the Training Sub-Directory
path = os.path.join(open_file + "/" + training_directory_name + "/"+directory_name, "Training_imgs_directory")
try:
    os.mkdir(path)
except OSError as error:
    print(error)
    
    
# Creating the Training Sub-Directory
path = os.path.join(open_file + "/" + training_directory_name + "/"+directory_name, "Test_imgs_directory")
try:
    os.mkdir(path)
except OSError as error:
    print(error)
    
# Creating the Sub-Directory for any other processing required
path = os.path.join(open_file + "/" + training_directory_name + "/"+directory_name, "Processing")
try:
    os.mkdir(path)
except OSError as error:
    print(error)

### Step 2. Preparing the Training Data

In order to obtain results from LOCATE, we need to prepare some files which make use of the data that we already have present, but some additional pre processing on it to generate intermediary files that will be used to train LOCATE.

The required images are:

1. The base image modality used in BIANCA ---
'(subject_name)-feature-(base_modality_name).nii.gz'

In [24]:
print(training_subject_bianca.replace("_","-"))

ms-013-tp1


In [31]:
print(f"cd {directory_1}/{training_directory_name} && cp {folder_name}/{file} {directory_name}/{training_folder}/{folder_name_fixed}_feature_FLAIR.nii.gz")

cd /Volumes/MS_training/Automated\ BIANCA\ Training/All_Files/Training-2022-12-04 && cp ms_013_tp1/ms_013_tp1_flair_brain.nii.gz LOCATE_Training-2022-12-04/Test_imgs_directory/ms-013-tp1_feature_FLAIR.nii.gz


In [32]:
for file in flairs:

    folder_name = "_".join(file.split('_')[0:3])
    
    if(folder_name!=training_subject_bianca):
        training_folder = "Training_imgs_directory"
    else:
        training_folder = "Test_imgs_directory"
    
    folder_name_fixed = "-".join(file.split('_')[0:3])
    p = os.popen(
    f"cd {directory_1}/{training_directory_name} && cp {folder_name}/{file} {directory_name}/{training_folder}/{folder_name_fixed}_feature_FLAIR.nii.gz")

    if p:
        output = p.read()
        print(output)  
        

The next file is optional, and since we are not making use of T1 Images or the MNI Space Registered Images, we will be skipping this part.

2. Any other additional images that were used as intensity features in BIANCA (--
featuresubset) (optional)
<subject_name>_feature_<modality_name>.nii.gz
(eg. <subject_name>_feature_T1.nii.gz, please note that it is mandatory to add
_feature_ in the filename before the modality name for it to be considered as a
feature)
__(OPTIONAL)__

As we had manually labelled lesion masks for the training of BIANCA, we will be using them in this step as well:

3. Lesion Manual mask (for training subjects only): binary mask (values of 0 and 1) indicating lesion voxels, based on manual segmentation --- (subject_name)_manualmask.nii.gz

In [34]:
training_folder = "Training_imgs_directory"
for mask in masks:

    folder_name = "_".join(mask.split('_')[0:3])
    folder_name_fixed = "-".join(mask.split('_')[0:3])
    
    if(folder_name!=training_subject_bianca):
        p = os.popen(
        f"cd {directory_1}/{training_directory_name} && cp {folder_name}/{mask} {directory_name}/{training_folder}/{folder_name_fixed}_manualmask.nii.gz")

        if p:
            output = p.read()
            print(output)  


4. Ventricle distance map: image where each voxel intensity represents the distance
from ventricles within the brain mask (this can be calculated using the FSL tool
distancemap – see the wiki for more details, example call: distancemap -i
<ventricle_mask_image_in_FLAIR_space> -m <brain_mask_in_FLAIR_space> -o
<subject_name>_ventdistmap.nii.gz)
 <subject_name>_ventdistmap.nii.gz

In [69]:
# Run FSL Fast to get the CSF pve file
for file in flairs:
    
    folder_name = "_".join(file.split('_')[0:3])
    
    if(folder_name!=training_subject_bianca):
        training_folder = "Training_imgs_directory"
    else:
        training_folder = "Test_imgs_directory"
    
    folder_name_fixed = "-".join(file.split('_')[0:3])
    
    print(f"cd {directory_1}/{training_directory_name} && /Applications/fsl/bin/fast -t 1 -n 3 -H 0.1 -I 4 -l 20.0 -o {directory_name}/Processing {directory_name}/{training_folder}/{folder_name_fixed}_feature_FLAIR.nii.nii.gz")
    print()
    print(f"cd {directory_1}/{training_directory_name} && cp {directory_name}/Processing/_pve_0.nii.gz {directory_name}/{training_folder}/{folder_name_fixed}_csf_pve.nii.gz")
    print()
    
    p = os.popen(
    f"cd {directory_1}/{training_directory_name} && /Applications/fsl/bin/fast -t 1 -n 3 -H 0.1 -I 4 -l 20.0 -o {directory_name}/Processing/ {directory_name}/{training_folder}/{folder_name_fixed}_feature_FLAIR.nii.nii.gz")

    if p:
        output = p.read()
        print(output)  
    

    p = os.popen(
    f"cd {directory_1}/{training_directory_name} && cp {directory_name}/Processing/_pve_0.nii.gz {directory_name}/{training_folder}/{folder_name_fixed}_csf_pve.nii.gz")

    if p:
        output = p.read()
        print(output) 

cd /Volumes/MS_training/Automated\ BIANCA\ Training/All_Files/Training-2022-12-04 && /Applications/fsl/bin/fast -t 1 -n 3 -H 0.1 -I 4 -l 20.0 -o LOCATE_Training-2022-12-04/Processing LOCATE_Training-2022-12-04/Training_imgs_directory/ms-002-tp1_feature_FLAIR.nii.nii.gz

cd /Volumes/MS_training/Automated\ BIANCA\ Training/All_Files/Training-2022-12-04 && cp LOCATE_Training-2022-12-04/Processing/_pve_0.nii.gz LOCATE_Training-2022-12-04/Training_imgs_directory/ms-002-tp1_csf_pve.nii.gz



cd /Volumes/MS_training/Automated\ BIANCA\ Training/All_Files/Training-2022-12-04 && /Applications/fsl/bin/fast -t 1 -n 3 -H 0.1 -I 4 -l 20.0 -o LOCATE_Training-2022-12-04/Processing LOCATE_Training-2022-12-04/Training_imgs_directory/ms-003-tp1_feature_FLAIR.nii.nii.gz

cd /Volumes/MS_training/Automated\ BIANCA\ Training/All_Files/Training-2022-12-04 && cp LOCATE_Training-2022-12-04/Processing/_pve_0.nii.gz LOCATE_Training-2022-12-04/Training_imgs_directory/ms-003-tp1_csf_pve.nii.gz



cd /Volumes/MS_tra

flirt -ref MNI152_2mm_brain.nii.gz -in ms-002-tp1_feature_FLAIR.nii.gz -omat my_affine_transf.mat


fnirt --in=ms-002-tp1_feature_FLAIR.nii.gz --aff=my_affine_transf.mat --cout=my_nonlinear_transf --config=T1_2_MNI152_2mm

applywarp --ref=MNI152_2mm_brain.nii.gz --in=ms-002-tp1_feature_FLAIR.nii.gz --warp=my_nonlinear_transf --out=my_warped_structural

In [ ]:
# Obtain the warp_file_MNI2structural


In [ ]:
# # distancemap 
# # -i <ventricle_mask_image_in_FLAIR_space> 
# # -m <brain_mask_in_FLAIR_space> 
# # -o <subject_name>_ventdistmap.nii.gz)

# We need to obtain the ventricle mask image in FLAIR Space

#### BIANCA Output for all files
From the above training of BIANCA, we obtained some training files which we are going to use to generate bianca outputs for all of the files that we used for the training

5. BIANCA output: unthresholded lesion probability map (LPM) obtained from
BIANCA
(subject_name)_BIANCA_LPM.nii.gz

In [37]:
file_path = (f"{open_file}/{training_directory_name}").replace(" ", "\ ")
classifier = file_path + "/mytraining"
print("Classifier: ",classifier)
print()

for file in flairs:
    
    folder_name = "-".join(file.split('_')[0:3])

    masterfile = f"{open_file}/{training_directory_name}/{directory_name}/Processing/{folder_name}_masterfile.txt"

    print(f"{directory_name}/Training_imgs_directory/{folder_name}_feature_FLAIR.nii.gz")
    print()
    file = open(masterfile, 'w')
    file.write(f"{directory_name}/Training_imgs_directory/{folder_name}_feature_FLAIR.nii.gz")
    file.close()
    
    masterfile=masterfile.replace(" ", "\ ")
    bianca_output = f"{folder_name}_BIANCA_LPM.nii.gz"
    
    print(f"cd {file_path} && bianca --singlefile={masterfile} --loadclassifierdata={classifier} --querysubjectnum=1 --brainmaskfeaturenum=1 -o {directory_name}/Training_imgs_directory/{bianca_output} -v")
    print()
    
    p = os.popen(
        f"cd {file_path} && bianca --singlefile={masterfile} --loadclassifierdata={classifier} --querysubjectnum=1 --brainmaskfeaturenum=1 -o {directory_name}/Training_imgs_directory/{bianca_output} -v")
    if p:
        output = p.read()
        print(output)
    

Classifier:  /Volumes/MS_training/Automated\ BIANCA\ Training/All_Files/Training-2022-12-04/mytraining

LOCATE_Training-2022-12-04/Training_imgs_directory/ms-002-tp1_feature_FLAIR.nii.gz

cd /Volumes/MS_training/Automated\ BIANCA\ Training/All_Files/Training-2022-12-04 && bianca --singlefile=/Volumes/MS_training/Automated\ BIANCA\ Training/All_Files/Training-2022-12-04/LOCATE_Training-2022-12-04/Processing/ms-002-tp1_masterfile.txt --loadclassifierdata=/Volumes/MS_training/Automated\ BIANCA\ Training/All_Files/Training-2022-12-04/mytraining --querysubjectnum=1 --brainmaskfeaturenum=1 -o LOCATE_Training-2022-12-04/Training_imgs_directory/ms-002-tp1_BIANCA_LPM.nii.gz -v



/Applications/fsl/bin/bianca:558: DeprecationWarning: get_header method is deprecated.
Please use the ``img.header`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  GlobalOpts.pixdims=imobj.get_header().get_zooms()
/Applications/fsl/bin/bianca:1026: DeprecationWarning: get_affine method is deprecated.
Please use the ``img.affine`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  pnii = nib.Nifti1Image(pvalim,queryimcl.get_affine(),header=queryimcl.get_header())
/Applications/fsl/bin/bianca:1026: DeprecationWarning: get_header method is deprecated.
Please use the ``img.header`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  pnii = nib.Nifti1Image(pvalim,queryimcl.get_affine(),header=queryimcl.get_header())


Number of modalities = 1 , number of possible training subjects = 1
Files are: label file list = , data file list = ['/Volumes/MS_training/Automated BIANCA Training/All_Files/Training-2022-12-04/LOCATE_Training-2022-12-04/Processing/ms-002-tp1_masterfile.txt']
Number of training points = 2000, mode = []
no spatial features used
Using all available images as intensity features (no intensity features subset specified)
Loading training data  /Volumes/MS_training/Automated BIANCA Training/All_Files/Training-2022-12-04/mytraining
Training data is size (181166, 1)
Loading query data
  Query file list = ['LOCATE_Training-2022-12-04/Training_imgs_directory/ms-002-tp1_feature_FLAIR.nii.gz']
 ... reading image LOCATE_Training-2022-12-04/Training_imgs_directory/ms-002-tp1_feature_FLAIR.nii.gz
Training classifier
Applying classifier
Calculating p-values
Saving output image

LOCATE_Training-2022-12-04/Training_imgs_directory/ms-003-tp1_feature_FLAIR.nii.gz

cd /Volumes/MS_training/Automated\ BIANCA

/Applications/fsl/bin/bianca:558: DeprecationWarning: get_header method is deprecated.
Please use the ``img.header`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  GlobalOpts.pixdims=imobj.get_header().get_zooms()
/Applications/fsl/bin/bianca:1026: DeprecationWarning: get_affine method is deprecated.
Please use the ``img.affine`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  pnii = nib.Nifti1Image(pvalim,queryimcl.get_affine(),header=queryimcl.get_header())
/Applications/fsl/bin/bianca:1026: DeprecationWarning: get_header method is deprecated.
Please use the ``img.header`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  pnii = nib.Nifti1Image(pvalim,queryimcl.get_affine(),header=queryimcl.get_header())


Number of modalities = 1 , number of possible training subjects = 1
Files are: label file list = , data file list = ['/Volumes/MS_training/Automated BIANCA Training/All_Files/Training-2022-12-04/LOCATE_Training-2022-12-04/Processing/ms-003-tp1_masterfile.txt']
Number of training points = 2000, mode = []
no spatial features used
Using all available images as intensity features (no intensity features subset specified)
Loading training data  /Volumes/MS_training/Automated BIANCA Training/All_Files/Training-2022-12-04/mytraining
Training data is size (181166, 1)
Loading query data
  Query file list = ['LOCATE_Training-2022-12-04/Training_imgs_directory/ms-003-tp1_feature_FLAIR.nii.gz']
 ... reading image LOCATE_Training-2022-12-04/Training_imgs_directory/ms-003-tp1_feature_FLAIR.nii.gz
Training classifier
Applying classifier
Calculating p-values
Saving output image

LOCATE_Training-2022-12-04/Training_imgs_directory/ms-003-tp6_feature_FLAIR.nii.gz

cd /Volumes/MS_training/Automated\ BIANCA

...

/Applications/fsl/bin/bianca:558: DeprecationWarning: get_header method is deprecated.
Please use the ``img.header`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  GlobalOpts.pixdims=imobj.get_header().get_zooms()
/Applications/fsl/bin/bianca:1026: DeprecationWarning: get_affine method is deprecated.
Please use the ``img.affine`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  pnii = nib.Nifti1Image(pvalim,queryimcl.get_affine(),header=queryimcl.get_header())
/Applications/fsl/bin/bianca:1026: DeprecationWarning: get_header method is deprecated.
Please use the ``img.header`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  pnii = nib.Nifti1Image(pvalim,queryimcl.get_affine(),header=queryimcl.get_header())


Number of modalities = 1 , number of possible training subjects = 1
Files are: label file list = , data file list = ['/Volumes/MS_training/Automated BIANCA Training/All_Files/Training-2022-12-04/LOCATE_Training-2022-12-04/Processing/ms-011-tp1_masterfile.txt']
Number of training points = 2000, mode = []
no spatial features used
Using all available images as intensity features (no intensity features subset specified)
Loading training data  /Volumes/MS_training/Automated BIANCA Training/All_Files/Training-2022-12-04/mytraining
Training data is size (181166, 1)
Loading query data
  Query file list = ['LOCATE_Training-2022-12-04/Training_imgs_directory/ms-011-tp1_feature_FLAIR.nii.gz']
 ... reading image LOCATE_Training-2022-12-04/Training_imgs_directory/ms-011-tp1_feature_FLAIR.nii.gz
Training classifier
Applying classifier
Calculating p-values
Saving output image

LOCATE_Training-2022-12-04/Training_imgs_directory/ms-012-tp1_feature_FLAIR.nii.gz

cd /Volumes/MS_training/Automated\ BIANCA

/Applications/fsl/bin/bianca:558: DeprecationWarning: get_header method is deprecated.
Please use the ``img.header`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  GlobalOpts.pixdims=imobj.get_header().get_zooms()
/Applications/fsl/bin/bianca:1026: DeprecationWarning: get_affine method is deprecated.
Please use the ``img.affine`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  pnii = nib.Nifti1Image(pvalim,queryimcl.get_affine(),header=queryimcl.get_header())
/Applications/fsl/bin/bianca:1026: DeprecationWarning: get_header method is deprecated.
Please use the ``img.header`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  pnii = nib.Nifti1Image(pvalim,queryimcl.get_affine(),header=queryimcl.get_header())


Number of modalities = 1 , number of possible training subjects = 1
Files are: label file list = , data file list = ['/Volumes/MS_training/Automated BIANCA Training/All_Files/Training-2022-12-04/LOCATE_Training-2022-12-04/Processing/ms-012-tp1_masterfile.txt']
Number of training points = 2000, mode = []
no spatial features used
Using all available images as intensity features (no intensity features subset specified)
Loading training data  /Volumes/MS_training/Automated BIANCA Training/All_Files/Training-2022-12-04/mytraining
Training data is size (181166, 1)
Loading query data
  Query file list = ['LOCATE_Training-2022-12-04/Training_imgs_directory/ms-012-tp1_feature_FLAIR.nii.gz']
 ... reading image LOCATE_Training-2022-12-04/Training_imgs_directory/ms-012-tp1_feature_FLAIR.nii.gz
Training classifier
Applying classifier
Calculating p-values
Saving output image

LOCATE_Training-2022-12-04/Training_imgs_directory/ms-013-tp1_feature_FLAIR.nii.gz

cd /Volumes/MS_training/Automated\ BIANCA

Traceback (most recent call last):
  File "/Applications/fsl/fslpython/envs/fslpython/lib/python3.7/site-packages/nibabel/loadsave.py", line 40, in load
    stat_result = os.stat(filename)
FileNotFoundError: [Errno 2] No such file or directory: 'LOCATE_Training-2022-12-04/Training_imgs_directory/ms-013-tp1_feature_FLAIR.nii.gz'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Applications/fsl/bin/bianca", line 1033, in <module>
    p_main()
  File "/Applications/fsl/bin/bianca", line 999, in p_main
    [check_dat,querypts,_] = load_vox_data([queryfilelist],matfilelist=[querymat],labelfilelist=querylabel,mode=['allpoints'],spatial_features=GlobalOpts.spatial_features,use_labels=False)
  File "/Applications/fsl/bin/bianca", line 557, in load_vox_data
    imobj = nib.load(filelist[GlobalOpts.maskfeaturenum])
  File "/Applications/fsl/fslpython/envs/fslpython/lib/python3.7/site-packages/nibabel/loadsave.py", line 42, in load


#### For the Testing File

In [44]:
file = flairs[len(flairs)-1]

folder_name = "-".join(file.split('_')[0:3])

masterfile = f"{open_file}/{training_directory_name}/{directory_name}/Processing/{folder_name}_masterfile.txt"

print(f"{directory_name}/Test_imgs_directory/{folder_name}_feature_FLAIR.nii.gz")
print()
file = open(masterfile, 'w')
file.write(f"{directory_name}/Test_imgs_directory/{folder_name}_feature_FLAIR.nii.gz")
file.close()

masterfile=masterfile.replace(" ", "\ ")
bianca_output = f"{folder_name}_BIANCA_LPM.nii.gz"

print(f"cd {file_path} && bianca --singlefile={masterfile} --loadclassifierdata={classifier} --querysubjectnum=1 --brainmaskfeaturenum=1 -o {directory_name}/Test_imgs_directory/{bianca_output} -v")
print()

p = os.popen(
    f"cd {file_path} && bianca --singlefile={masterfile} --loadclassifierdata={classifier} --querysubjectnum=1 --brainmaskfeaturenum=1 -o {directory_name}/Test_imgs_directory/{bianca_output} -v")
if p:
    output = p.read()
    print(output)

LOCATE_Training-2022-12-04/Test_imgs_directory/ms-013-tp1_feature_FLAIR.nii.gz

cd /Volumes/MS_training/Automated\ BIANCA\ Training/All_Files/Training-2022-12-04 && bianca --singlefile=/Volumes/MS_training/Automated\ BIANCA\ Training/All_Files/Training-2022-12-04/LOCATE_Training-2022-12-04/Processing/ms-013-tp1_masterfile.txt --loadclassifierdata=/Volumes/MS_training/Automated\ BIANCA\ Training/All_Files/Training-2022-12-04/mytraining --querysubjectnum=1 --brainmaskfeaturenum=1 -o LOCATE_Training-2022-12-04/Test_imgs_directory/ms-013-tp1_BIANCA_LPM.nii.gz -v

Number of modalities = 1 , number of possible training subjects = 1
Files are: label file list = , data file list = ['/Volumes/MS_training/Automated BIANCA Training/All_Files/Training-2022-12-04/LOCATE_Training-2022-12-04/Processing/ms-013-tp1_masterfile.txt']
Number of training points = 2000, mode = []
no spatial features used
Using all available images as intensity features (no intensity features subset specified)
Loading traini

/Applications/fsl/bin/bianca:558: DeprecationWarning: get_header method is deprecated.
Please use the ``img.header`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  GlobalOpts.pixdims=imobj.get_header().get_zooms()
/Applications/fsl/bin/bianca:1026: DeprecationWarning: get_affine method is deprecated.
Please use the ``img.affine`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  pnii = nib.Nifti1Image(pvalim,queryimcl.get_affine(),header=queryimcl.get_header())
/Applications/fsl/bin/bianca:1026: DeprecationWarning: get_header method is deprecated.
Please use the ``img.header`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  pnii = nib.Nifti1Image(pvalim,queryimcl.get_affine(),header=queryimcl.get_header())


Using a simple fslmaths command, we will obtain a mask for the Extracted Brain images that we already have

6. Brain mask: binary mask obtained from FSL-BET or any other method
<subject_name>_brainmask.nii.gz

In [46]:
for file in flairs:
    
    folder_name = "_".join(file.split('_')[0:3])
    
    if(folder_name!=training_subject_bianca):
        training_folder = "Training_imgs_directory"
    else:
        training_folder = "Test_imgs_directory"
    
    folder_name_fixed = "-".join(file.split('_')[0:3])
    
    print(f"cd {file_path}/{directory_name}/{training_folder} && fslmaths {folder_name_fixed}_feature_FLAIR.nii.gz -bin {folder_name_fixed}_brainmask.nii.gz")
    p = os.popen(
        f"cd {file_path}/{directory_name}/{training_folder} && fslmaths {folder_name_fixed}_feature_FLAIR.nii.gz -bin {folder_name_fixed}_brainmask.nii.gz")
    if p:
        output = p.read()
        print(output)

cd /Volumes/MS_training/Automated\ BIANCA\ Training/All_Files/Training-2022-12-04/LOCATE_Training-2022-12-04/Training_imgs_directory && fslmaths ms-002-tp1_feature_FLAIR.nii.gz -bin ms-002-tp1_brainmask.nii.gz

cd /Volumes/MS_training/Automated\ BIANCA\ Training/All_Files/Training-2022-12-04/LOCATE_Training-2022-12-04/Training_imgs_directory && fslmaths ms-003-tp1_feature_FLAIR.nii.gz -bin ms-003-tp1_brainmask.nii.gz

cd /Volumes/MS_training/Automated\ BIANCA\ Training/All_Files/Training-2022-12-04/LOCATE_Training-2022-12-04/Training_imgs_directory && fslmaths ms-003-tp6_feature_FLAIR.nii.gz -bin ms-003-tp6_brainmask.nii.gz

cd /Volumes/MS_training/Automated\ BIANCA\ Training/All_Files/Training-2022-12-04/LOCATE_Training-2022-12-04/Training_imgs_directory && fslmaths ms-004-tp1_feature_FLAIR.nii.gz -bin ms-004-tp1_brainmask.nii.gz

cd /Volumes/MS_training/Automated\ BIANCA\ Training/All_Files/Training-2022-12-04/LOCATE_Training-2022-12-04/Training_imgs_directory && fslmaths ms-004-tp7_

7. BIANCA mask: binary mask, as obtained from make_bianca_mask (white matter
mask excluding sub-cortical regions) - If you are not using BIANCA mask in your
analysis, make a copy of the brain mask and rename it as BIANCA mask.
 <subject_name>_biancamask.nii.gz